# 101 Asynchronous programming with Python
## Module 1

### Agenda:

* Key definitions.
  * Concurrency, multitasking, 
  * Coroutines.
* The Python syntax: what means "async" and why you need "await".
* What is event loop and why you should not care about it. 


![whooper swans on snow](../images/whooper swans on snow.jpg)

**Concurrent computing** is a form of computing in which several computations are executed concurrently—during overlapping time periods—instead of sequentially, with one completing before the next starts.

<div align="right">
    – <a href="https://en.wikipedia.org/wiki/Concurrent_computing">Wikipedia / Concurrent_computing </a>
</div>



